## Installation



In [ ]:
pip install wandb numpy pandas matplotlib torch torchvision

## Dataset

In [1]:
# from google.colab import drive
# drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
#!unzip /content/drive/MyDrive/DL_Assignment2/Dataset/nature_12K.zip -d /content/drive/MyDrive/DL_Assignment2/Dataset

## Libraries

In [1]:
import torch
import os
from torchvision import datasets, transforms
from sklearn.model_selection import train_test_split, StratifiedShuffleSplit
from torch.utils.data import Subset, DataLoader
import numpy as np
import torch.nn as nn
import torch.optim as optim
import wandb
import tqdm

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

cuda


## Dataset loader

In [2]:
def validationDataSplit(train_dataset):
  classLabels = [label for _,label in train_dataset.samples]
  num_classes = len(np.unique(classLabels))

  sss = StratifiedShuffleSplit(n_splits=1, test_size=0.2, random_state=42)
  train_indices, val_indices = next(sss.split(train_dataset.samples, classLabels))

  train_subset = Subset(train_dataset, train_indices)
  val_subset = Subset(train_dataset, val_indices)
  return train_subset, val_subset, num_classes


def load_data(base_dir, isDataAug):
  train_dir = os.path.join(base_dir, 'train')
  test_dir = os.path.join(base_dir, 'val')

  train_transform, test_transform = None, None

  if isDataAug == False:
    train_transform = transforms.Compose([
      transforms.Resize(256),
      transforms.CenterCrop(224),
      transforms.ToTensor(),
      transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])])
  else:
    train_transform = transforms.Compose([
      transforms.Resize(256),
      transforms.CenterCrop(224),
      transforms.RandomHorizontalFlip(),
      transforms.RandomRotation(10),
      transforms.ToTensor(),
      transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])])

  test_transform = transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])])

  train_dataset = datasets.ImageFolder(train_dir, transform=train_transform)
  test_dataset = datasets.ImageFolder(test_dir, transform=test_transform)
  train_dataset, val_dataset, num_classes = validationDataSplit(train_dataset)

  # print(f"inp: {train_dataset[0][0].shape} {train_dataset[0][1]}")

  train_loader = DataLoader(train_dataset,shuffle=True,num_workers=2,batch_size=64,pin_memory=True)
  test_loader = DataLoader(test_dataset,shuffle=True,num_workers=2,batch_size=64,pin_memory=True)
  val_loader = DataLoader(val_dataset,shuffle=True,num_workers=2,batch_size=64,pin_memory=True)

  return train_loader, test_loader, val_loader, num_classes

# load_data("/content/drive/MyDrive/DL_Assignment2/Dataset/inaturalist_12K/", True)

## Network

In [3]:
class ConvolutionalNeuralNetwork(nn.Module):
  activationFunctionsMap = {"ReLU": nn.ReLU, "GELU": nn.GELU, "SiLU": nn.SiLU}
  # optimizersMap = {"sgd": optim.SGD, "rmsprop": optim.RMSprop, "adam": optim.Adam}

  def __init__(self, num_classes,
               num_filters, filter_sizes,
               activationFun, optimizer,
               n_neurons_denseLayer,
               isBatchNormalization, dropout,
               learning_rate=0.001,
               momentum=0.5, beta = 0.9,
               beta1=0.9, beta2=0.99,
               epsilon=1e-8, weight_decay=0.0001):
    super(ConvolutionalNeuralNetwork, self).__init__()
    self.num_classes = num_classes
    self.num_filters = num_filters
    self.filter_sizes = filter_sizes
    self.activationFun = ConvolutionalNeuralNetwork.activationFunctionsMap[activationFun]
    # self.optimizer = ConvolutionalNeuralNetwork.optimizersMap[optimizer]

    self.n_neurons_denseLayer = n_neurons_denseLayer
    self.isBatchNormalization = isBatchNormalization
    self.dropout = dropout

    self.lr = learning_rate
    self.momentum = momentum
    self.betas = (beta1, beta2)
    self.eps = epsilon
    self.alpha = beta
    self.weight_decay = weight_decay

    self.defineModel()

    if(optimizer == "sgd"):
      self.optimizer = optim.SGD(self.parameters(), lr=self.lr, momentum=self.momentum, weight_decay=self.weight_decay)
    elif(optimizer == "rmsprop"):
      self.optimizer = optim.RMSprop(self.parameters(), lr=self.lr, alpha=self.alpha, eps=self.eps, weight_decay=self.weight_decay)
    elif(optimizer == "adam"):
      self.optimizer = optim.Adam(self.parameters(), lr=self.lr, betas=self.betas, eps=self.eps, weight_decay=self.weight_decay)



  def defineModel(self):
    self.model = nn.Sequential()

    inChannels = 3;     # RGB channels for inaturalist
    for i in range(len(self.num_filters)):
      self.model.append(nn.Conv2d(inChannels, self.num_filters[i], self.filter_sizes[i], padding=self.filter_sizes[i]//2))
      if self.isBatchNormalization:
        self.model.append(nn.BatchNorm2d(self.num_filters[i]))
      self.model.append(self.activationFun())
      self.model.append(nn.MaxPool2d(kernel_size=2))
      inChannels = self.num_filters[i]

    # computing flattened size
    input_shape = (3, 224, 224)
    with torch.no_grad():
      dummy_input = torch.zeros(1, *input_shape)
      dummy_output = self.model(dummy_input)
      flattened_size = dummy_output.view(dummy_output.size(0), -1).size(1)

    self.model.append(nn.Flatten())
    self.model.append(nn.Linear(flattened_size, self.n_neurons_denseLayer))
    self.model.append(self.activationFun())

    if(self.dropout > 0):
      self.model.append(nn.Dropout(self.dropout))

    self.model.append(nn.Linear(self.n_neurons_denseLayer, self.num_classes))

  def forward(self, inputs):
    return self.model(inputs)

  def backward(self, outputs, labels):
    loss = nn.CrossEntropyLoss()(outputs, labels)
    loss.backward()

  def updateWeights(self):
    self.optimizer.step()

## Training CNN

In [8]:
sweep_configuration = {
    "method": "bayes",
    "name" : "train_sweep",
    "metric": {"name": "val_accuracy", "goal": "maximize"},
    "parameters": {
        "num_filters": {'values': [[32, 32, 32, 32, 32], [32, 64, 64, 128, 256], [256, 128, 64, 64, 32]]},
        "filter_sizes": {'values': [[3, 3, 3, 3, 3], [5, 5, 5, 5, 5]]},
        "activation": {"values": ["ReLU", "SiLU", "GELU"]},
        "optimizer": {"values": ["adam", "rmsprop", "sgd"]},
        "learning_rate": {"values": [1e-3]},
        "weight_decay": {"values": [0.0001]},
        "momentum": {"values": [0.9]},
        "beta": {"values": [0.9]},
        "beta1": {"values":[0.9]},
        "beta2": {"values": [0.999]},
        "epsilon": {"values": [1e-8]},
        # "base_dir": {"values":["/content/drive/MyDrive/DL_Assignment2/Dataset/inaturalist_12K/"]},
        "base_dir": {"values": ["/kaggle/input/inaturalist/inaturalist_12K"]},
        "isDataAug": {"values": ["False", "True"]},
        "isBatchNormalization": {"values": ["True", "False"]},
        "dropout": {"values": [0.2, 0.3]},
        "n_neurons_denseLayer": {"values": [128]}
    }
}


def findOutputs(cnn, inputDataLoader):
  cnn.eval()  # setting the model to evaluation model
  outputs = []
  total_loss = 0.0
  n_correct = 0
  n_samples = 0

  with torch.no_grad():
    for batch_idx, (x_batch, y_batch) in enumerate(inputDataLoader):
      x_batch, y_batch = x_batch.to(device), y_batch.to(device)
      batch_outputs = cnn(x_batch)

      loss = nn.CrossEntropyLoss()(batch_outputs, y_batch)
      total_loss += loss.item() * x_batch.size(0)

      y_pred_batch = torch.argmax(batch_outputs, dim=1)
      n_correct += (y_pred_batch == y_batch).sum().item()
      n_samples += x_batch.size(0)

      outputs.append(batch_outputs)

  outputs = torch.cat(outputs)
  accuracy = (n_correct * 100.0) / n_samples
  avg_loss = total_loss / n_samples
  return outputs, accuracy, avg_loss

def trainNeuralNetwork_sweep():
  wandb.init(mode="online")
  args = wandb.config
  train_loader, test_loader, val_loader, num_classes = load_data(args["base_dir"], args["isDataAug"])
  activationFun = args["activation"]
  optimizer = args["optimizer"]
  learning_rate = args["learning_rate"]
  momentum = args["momentum"]
  beta = args["beta"]
  beta1 = args["beta1"]
  beta2 = args["beta2"]
  epsilon = args["epsilon"]
  weight_decay = args["weight_decay"]
  dropout = args["dropout"]
  num_filters = args["num_filters"]
  filter_sizes = args["filter_sizes"]
  n_neurons_denseLayer = args["n_neurons_denseLayer"]
  isBatchNormalization = args["isBatchNormalization"]
  isDataAug = args["isDataAug"]

  wandb.run.name = f"{activationFun}_{optimizer}_{dropout}_{n_neurons_denseLayer}_DataAug-{isDataAug}_BatchNorm-{isBatchNormalization}"
  best_val_accuracy = 0.0
  best_accuracy_epoch = -1

  cnn = ConvolutionalNeuralNetwork(num_classes,
                                   num_filters, filter_sizes,
                                   activationFun, optimizer,
                                   n_neurons_denseLayer,
                                   isBatchNormalization, dropout,
                                   learning_rate,
                                   momentum, beta,
                                   beta1, beta2,
                                   epsilon, weight_decay)
  cnn.to(device)

  epochs = 10
  for epochNum in range(epochs):
    print(f"Epoch {epochNum}:")
    for batch_idx, (x_batch, y_batch) in enumerate(train_loader):
      if(batch_idx % 40 == 0):
        print(f"Batch idx {batch_idx} running")
      x_batch, y_batch = x_batch.to(device), y_batch.to(device)
      cnn.optimizer.zero_grad()
      outputs = cnn(x_batch)
      cnn.backward(outputs, y_batch)
      cnn.updateWeights()

    # Validation accuracy
    val_outputs, val_accuracy, val_loss = findOutputs(cnn, val_loader)
    wandb.run.summary["metric_name"] = val_accuracy
    print(f"validation: loss={val_loss}, accuracy={val_accuracy}")

    # Train accuracy
    train_outputs, train_accuracy, train_loss = findOutputs(cnn, train_loader)
    print(f"training: loss={train_loss}, training={train_accuracy}")

    if val_accuracy > best_val_accuracy:
      best_val_accuracy = val_accuracy
      best_accuracy_epoch = epochNum
      wandb.run.summary["best_val_accuracy"] = best_val_accuracy
      wandb.run.summary["best_accuracy_epoch"] = best_accuracy_epoch

    wandb.log({
        "epoch": epochNum + 1,
        "val_loss": val_loss,
        "val_accuracy": val_accuracy,
        "train_loss": train_loss,
        "train_accuracy": train_accuracy
        },commit=True)
      
  wandb.log({
      "best_acc_epoch": best_accuracy_epoch,
      "best_val_accuracy": best_val_accuracy
  })
  del cnn
  torch.cuda.empty_cache()

  wandb.finish()

In [9]:
wandb.login(key="x")
wandb_id = wandb.sweep(sweep_configuration, project="DA6401_Assignment2")
wandb.agent(wandb_id, function=trainNeuralNetwork_sweep)

wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


Create sweep with ID: iey9548t
Sweep URL: https://wandb.ai/nikhithaa-iit-madras/DA6401_Assignment2/sweeps/iey9548t


wandb: Agent Starting Run: 8m01mlle with config:
wandb: 	activation: GELU
wandb: 	base_dir: /kaggle/input/inaturalist/inaturalist_12K
wandb: 	beta: 0.9
wandb: 	beta1: 0.9
wandb: 	beta2: 0.999
wandb: 	dropout: 0.2
wandb: 	epsilon: 1e-08
wandb: 	filter_sizes: [5, 5, 5, 5, 5]
wandb: 	isBatchNormalization: True
wandb: 	isDataAug: True
wandb: 	learning_rate: 0.001
wandb: 	momentum: 0.9
wandb: 	n_neurons_denseLayer: 128
wandb: 	num_filters: [32, 32, 32, 32, 32]
wandb: 	optimizer: sgd
wandb: 	weight_decay: 0.0001


Epoch 0:
Batch idx 0 running
Batch idx 40 running
Batch idx 80 running
Batch idx 120 running
validation: loss=2.1231721992492676, accuracy=25.65
training: loss=2.1231721992492676, training=25.65
Epoch 1:
Batch idx 0 running
Batch idx 40 running
Batch idx 80 running
Batch idx 120 running
validation: loss=2.0430052852630616, accuracy=25.3
training: loss=2.0430052852630616, training=25.3
Epoch 2:
Batch idx 0 running
Batch idx 40 running
Batch idx 80 running
Batch idx 120 running
validation: loss=1.998781997680664, accuracy=29.0
training: loss=1.998781997680664, training=29.0
Epoch 3:
Batch idx 0 running
Batch idx 40 running
Batch idx 80 running
Batch idx 120 running
validation: loss=1.9636973457336426, accuracy=29.35
training: loss=1.9636973457336426, training=29.35
Epoch 4:
Batch idx 0 running
Batch idx 40 running
Batch idx 80 running
Batch idx 120 running
validation: loss=1.9445332355499267, accuracy=31.65
training: loss=1.9445332355499267, training=31.65
Epoch 5:
Batch idx 0 running
Ba

epoch,▁▂▃▃▄▅▆▆▇█
train_accuracy,▁▂▃▄▄▆▆▆▇█
train_loss,█▆▅▅▄▃▃▃▂▁
validation_accuracy,▁▁▄▄▅▇▇█▇█
validation_loss,█▆▅▄▄▁▂▂▁▁
best_model_path,/kaggle/working/best...
best_val_accuracy,35.2
epoch,10
metric_name,35.2
train_accuracy,41.04263
train_loss,1.6927


...

epoch,▁▂▃▃▄▅▆▆▇█
train_accuracy,▁▂▅▅▅▆▆███
train_loss,█▆▄▄▄▃▃▁▂▁
validation_accuracy,▁▃▅▆▆▇▆▇▇█
validation_loss,█▅▄▃▃▂▃▁▂▁
best_model_path,/kaggle/working/best...
best_val_accuracy,35.65
epoch,10
metric_name,35.65
train_accuracy,39.2299
train_loss,1.73352


wandb: Agent Starting Run: cr86ffi3 with config:
wandb: 	activation: SiLU
wandb: 	base_dir: /kaggle/input/inaturalist/inaturalist_12K
wandb: 	beta: 0.9
wandb: 	beta1: 0.9
wandb: 	beta2: 0.999
wandb: 	dropout: 0.2
wandb: 	epsilon: 1e-08
wandb: 	filter_sizes: [5, 5, 5, 5, 5]
wandb: 	isBatchNormalization: True
wandb: 	isDataAug: True
wandb: 	learning_rate: 0.001
wandb: 	momentum: 0.9
wandb: 	n_neurons_denseLayer: 128
wandb: 	num_filters: [32, 64, 64, 128, 256]
wandb: 	optimizer: rmsprop
wandb: 	weight_decay: 0.0001


Epoch 0:
Batch idx 0 running
Batch idx 40 running
Batch idx 80 running
Batch idx 120 running
validation: loss=2.4753534870147704, accuracy=15.4
training: loss=2.4753534870147704, training=15.4
Epoch 1:
Batch idx 0 running
Batch idx 40 running
Batch idx 80 running
Batch idx 120 running
validation: loss=2.103593620300293, accuracy=23.15
training: loss=2.103593620300293, training=23.15
Epoch 2:
Batch idx 0 running
Batch idx 40 running
Batch idx 80 running
Batch idx 120 running
validation: loss=2.0029942207336426, accuracy=29.95
training: loss=2.0029942207336426, training=29.95
Epoch 3:
Batch idx 0 running
Batch idx 40 running
Batch idx 80 running
Batch idx 120 running
validation: loss=2.028902931213379, accuracy=29.95
training: loss=2.028902931213379, training=29.95
Epoch 4:
Batch idx 0 running
Batch idx 40 running
Batch idx 80 running
Batch idx 120 running
validation: loss=2.181956775665283, accuracy=30.5
training: loss=2.181956775665283, training=30.5
Epoch 5:
Batch idx 0 running
Batch 

wandb: Ctrl + C detected. Stopping sweep.


epoch,▁▂▃▅▆▇█
train_accuracy,▁▄▆▆▆██
train_loss,█▄▃▃▅▂▁
validation_accuracy,▁▄▇▇▇██
validation_loss,█▃▂▂▄▁▁
epoch,7
metric_name,32.35
train_accuracy,35.65446
train_loss,1.81513
validation_accuracy,32.35
validation_loss,1.90811


In [12]:
# import os
print(os.listdir('/kaggle/working'))

['wandb', '.virtual_documents', 'best_model.pth']
